In [3]:
! pip install keras-tuner

     |████████████████████████████████| 71kB 3.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=b6a6afb693c42c07b401f8b078e1bc31e806313176caa8d1f58fa221c1f00ce4
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=c3dd2d8cc897f541d37a5ec96188c943935b81a5187f9c7d1542ae1ebe8cc10a
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [5]:
fashion_mnist = keras.datasets.fashion_mnist

In [6]:
(train_images, train_lables), (test_images, test_lables) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images),28, 28, 1)

In [10]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.conv2D(
                                filters = hp.Int('conv_1_filter', min_value = 32, max_value = 128, step = 16),
                                kernel_size = hp.choise('conv_1_kernel', values = [3,5]),
                                activation = 'relu',
                                input_shape = (28, 28, 1)
                            ),
                            keras.layers.Conv2D(
                                filters = hp.Int('conv_2_filter', min_value = 32, max_value = 64, step = 16),
                                kernel_size = hp.choise('conv_2_kernel', values = [3, 5]),
                                activation = 'relu'
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step = 16),
                                activation = 'relu'
                            ),    
                            keras.layers.Dense(10, activation = 'softmax')   #output layer
                             
  ])
  model.compile(optimizer = keras.optimizars.Adam(hp.Choice('learning_rate', values = [le-2, le-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search = RandomSearch(build_model, 
                            objective = 'val_accuracy',
                            max_trials = 5, directory = 'output', project_name = "Mnist Fashion")

In [ ]:
tuner_search.search(train_images, train_lables, epochs = 3, validation_split = 0.1)

In [ ]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [ ]:
model.summary()